In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from arch import arch_model

prices = pd.read_csv("store.csv")

# prices.tail()

# gch=arch_model(prices[-200:]["Close"],p=1,q=0)

# res=gch.fit(update_freq=5,disp="off")

# res.plot()

prices["DateTime"]=pd.to_datetime(prices["DateTime"])
wind_size=10
# plt.plot(prices[-wind_size:]["DateTime"],prices[-wind_size:]["Close"])
# plt.show()

# forecast=res.forecast(horizon=5)

# forecast.variance


In [ ]:
means=prices["Close"].rolling(window=wind_size).mean()
std=prices["Close"].rolling(window=wind_size).std()
plt.plot(prices["DateTime"],prices["Close"],label='price')
plt.plot(prices["DateTime"],means,label='rolling average')
plt.plot(prices["DateTime"],std,label='rolling std. dev.')
plt.title("Mean and Standard Deviation Plot")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend(loc='best')
plt.show(block=False)

In [ ]:
from statsmodels.tsa.stattools import adfuller
aprices_test = adfuller(prices["Close"],autolag='AIC')
df_output = pd.Series(aprices_test[0:4],index=["Test Static",'p-value','#Lags Used','Number of Observations Used'])
for key,value in aprices_test[4].items():
    df_output['Critical Value (%s)'%key]=value
print(df_output)

In [ ]:
prices_lg = np.log(prices["Close"])
plt.plot(prices_lg)

In [ ]:
means_lg=prices_lg.rolling(window=wind_size).mean()
std_lg=prices_lg.rolling(window=wind_size).std()
plt.plot(prices["DateTime"],prices_lg,label='log_price')
plt.plot(prices["DateTime"],means_lg,label='lg_rolling average')
plt.plot(prices["DateTime"],std_lg,label='lg_rolling std. dev.')
plt.title("Mean and Standard Deviation Plot")
plt.xlabel("Date")
plt.ylabel("Log Price")
plt.legend(loc='best')
plt.show(block=False)

In [ ]:
mod_dat=prices_lg-means_lg
mod_dat.tail()
mod_dat.dropna(inplace=True)
mod_dat.head()

In [ ]:
def test_stat(mod_dat):
    means_lg=mod_dat.rolling(window=wind_size).mean()
    std_lg=mod_dat.rolling(window=wind_size).std()
    plt.plot(mod_dat,label='log_price')
    plt.plot(means_lg,label='lg_rolling average')
    plt.plot(std_lg,label='lg_rolling std. dev.')
    plt.title("Mean and Standard Deviation Plot")
    plt.xlabel("Date")
    plt.ylabel("Log Price")
    plt.legend(loc='best')
    plt.show(block=False)
    aprices_test = adfuller(mod_dat,autolag='AIC')
    df_output = pd.Series(aprices_test[0:4],index=["Test Static",'p-value','#Lags Used','Number of Observations Used'])
    for key,value in aprices_test[4].items():
        df_output['Critical Value (%s)'%key]=value
    print(df_output)

In [ ]:
test_stat(mod_dat)

In [ ]:
edwa=prices_lg.ewm(halflife=wind_size,min_periods=0,adjust=True).mean()
plt.plot(prices_lg)
plt.plot(edwa,color='red')

In [ ]:
minmovavexpdecayave = prices_lg-edwa
test_stat(minmovavexpdecayave)

In [ ]:
dlds = prices_lg-prices_lg.shift()
plt.plot(dlds)

In [ ]:
dlds.dropna(inplace=True)
test_stat(dlds)

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decom = seasonal_decompose(prices_lg,period=len(prices_lg)//5)

trend = decom.trend
seasonal = decom.seasonal
residual = decom.resid

plt.subplot(221)
plt.plot(prices_lg,label='Original')
plt.legend(loc='best')
plt.subplot(222)
plt.plot(trend,label='Trend')
plt.legend(loc='best')
plt.subplot(223)
plt.plot(seasonal,label='Seasonality')
plt.legend(loc='best')
plt.subplot(224)
plt.plot(residual,label='Residuals')
plt.legend(loc='best')


In [ ]:
declogdat = residual
declogdat.dropna(inplace=True)
test_stat(declogdat)

In [ ]:
from statsmodels.tsa.stattools import acf, pacf
acf_original = acf(dlds,nlags=20)
pacf_original = pacf(dlds,nlags=20,method='ols')

plt.subplot(211)
plt.plot(acf_original)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
plt.title('Auto-Correlation Function')
plt.subplot(212)
plt.plot(pacf_original)
plt.axhline(y=0,linestyle='--',color='gray')
plt.axhline(y=-1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
plt.axhline(y=1.96/np.sqrt(len(dlds)),linestyle='--',color='gray')
plt.title('Partial-Correlation Function')
plt.tight_layout()

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(prices_lg, order=(2,1,2))
model_fit = model.fit()
plt.plot(dlds)
plt.plot(model_fit.fittedvalues,color='red')


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
model = ARIMA(prices_lg, order=(2,1,2))
model_fit_MA = model.fit()
plt.plot(dlds)
plt.plot(model_fit_MA.fittedvalues,color='blue')
plt.title('RSS: %.4f'% sum((model_fit_MA.fittedvalues)-dlds)**2)
